# AutoGluon Time Series - Forecasting Quick Start
--------------
이 실습은 https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-quick-start.html 코드를 기반으로 SageMaker에서 학습하는 방법을 가이드하고자 만들었습니다. 

## 1. 필요한 패키지 설치 및 업데이트

## 2. 환경 설정

Sagemaker 학습에 필요한 기본적인 package를 import 합니다. <br>
[boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)는 AWS 리소스와 동작하는 python 클래스를 제공하며, HTTP API 호출을 숨기는 추상화 모델입니다. boto3를 통해 python에서 Amazon EC2 인스턴스, S3 버켓과 같은 AWS 리소스와 동작할 수 있습니다.<br>
[sagemaker python sdk](https://sagemaker.readthedocs.io/en/stable/)는 Amazon SageMaker에서 기계 학습 모델을 교육 및 배포하기 위한 오픈 소스 라이브러리입니다.<br>

In [ ]:
# !pip install --upgrade autogluon

In [ ]:
import matplotlib.pyplot as plt
import sagemaker
# import splitfolders

import os
import time
import warnings

# from smexperiments.experiment import Experiment
# from smexperiments.trial import Trial

import boto3
import numpy as np

# from tqdm import tqdm
from time import strftime

from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
role = get_execution_role()

In [ ]:
sagemaker.__version__

## 3. Dataset 준비

In [ ]:
# Basic data configuration is initialised and stored in the Data Preparation notebook
# ...We just retrieve it here:
%store -r
assert bucket, "Variable `bucket` missing from IPython store"
assert data_prefix, "Variable `data_prefix` missing from IPython store"

## 5. 실험 설정

학습 시 사용한 소스코드와 output 정보를 저장할 위치를 선정합니다. 이 값은 필수로 설정하지 않아도 되지만, 코드와 결과물을 S3에 저장할 때 체계적으로 정리하는데 활용할 수 있습니다.

In [ ]:
code_location = f's3://{bucket}/poc_autogluon/sm_codes'
output_path = f's3://{bucket}/poc_autogluon/output' 

실험에서 표준 출력으로 보여지는 metrics 값을 정규 표현식을 이용하여 SageMaker에서 값을 capture할 수 있습니다. 이 값은 필수로 설정하지 않아도 되지만, SageMaker Experiments에 Metrics 정보를 남길 수 있어서 실험 관리에 유용합니다.

In [ ]:
metric_definitions = [
    {'Name': 'Epoch', 'Regex': 'Epoch ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?):'},
    {'Name': 'Train Loss Step', 'Regex': 'train_loss_step=([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'},
    {'Name': 'Train Loss Epoch', 'Regex': 'train_loss_epoch=([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'},
    {'Name': 'valid_loss', 'Regex': 'valid_loss=([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'},
]

다양한 실험 조건을 테스트하기 위해 hyperparameters로 argument 값들을 노트북에서 설정할 수 있으며, 이 값은 학습 스크립트에서 argument인 변수로 받아서 활용이 가능합니다.

In [ ]:
hyperparameters = {
    'prediction_length' : 336,
    
}

분산학습과 spot 학습을 사용할지를 선정할 수 있습니다. <br>
분산학습의 경우 [SageMaker data parallel library](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel.html)를 사용하고자 할 경우 distribution을 아래와 같이 설정한 후 사용할 수 있습니다. 학습 스크립트는 분산 학습 Library로 구현이 필요합니다. <br>
[spot 학습](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html)을 사용하고자 할 경우 학습 파라미터에 spot 파라미터를 True로 변경한 다음, 자원이 없을 때 대기하는 시간인 max_wait (초)를 설정해야 합니다.

In [ ]:
experiment_name = 'autogluon-poc-exp1'
distribution = None
do_spot_training = True
max_wait = None
max_run = 4*60*60

if do_spot_training:
    max_wait=max_run

In [ ]:
instance_type="ml.g5.xlarge"
# instance_type='local_gpu'
instance_count=1

## 6. checkpoints 설정, 데이터 위치 설정, Local Mode 설정

### 6-2. checkpoints와 데이터 위치 설정, Local Mode 설정

In [ ]:
from pathlib import Path

source_dir = f'{Path.cwd()}/autogluon'
    
if instance_type =='local_gpu' or instance_type =='local':
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    inputs = f'file://{Path.cwd()}/data/amzforecast'

    do_spot_training = False
    checkpoint_s3_uri=None
    max_wait = None
else:
    sagemaker_session = sagemaker.Session()
    inputs = f's3://{bucket}/data/amzforecast'
    checkpoint_s3_uri = f's3://{bucket}/poc_autogluon|/checkpoints'

## 7. 학습을 위한 Estimator 선언

AWS 서비스 활용 시 role (역할) 설정은 매우 중요합니다. 이 노트북에서 사용하는 role은 노트북과 training job을 실행할 때 사용하는 role이며, role을 이용하여 다양한 AWS 서비스에 대한 접근 권한을 설정할 수 있습니다.

In [ ]:
# all input configurations, parameters, and metrics specified in estimator 
# definition are automatically tracked
estimator = PyTorch(
    entry_point='ag_main.py',
    source_dir=source_dir,
    role=role,
    sagemaker_session=sagemaker_session,
    framework_version='2.0',
    py_version='py310',
    instance_count=instance_count,
    instance_type=instance_type,
#     volume_size=256,
    code_location = code_location,
    output_path=output_path,
    hyperparameters=hyperparameters,
    # distribution=distribution,
    metric_definitions=metric_definitions,
    max_run=max_run,
    checkpoint_s3_uri=checkpoint_s3_uri,
    use_spot_instances=do_spot_training,
    max_wait=max_wait,
    disable_profiler=True,
    debugger_hook_config=False,
)

## 8. 학습 수행 - 시작

In [ ]:
create_date = strftime("%m%d-%H%M%s")
job_name = f"{experiment_name}-{create_date}"

# Now associate the estimator with the Experiment and Trial
estimator.fit(
    inputs={'training': inputs}, 
    job_name=job_name,
    wait=False
)

In [ ]:
job_name=estimator.latest_training_job.name

아래 명령어를 이용하여 시작된 학습에 대한 로그를 노트북에서 확인합니다. 이 로그는 CloudWatch에서도 확인이 가능합니다. <br> 
아래 명령어를 실행해도 학습이 시작되는 것이 아니며, 실행된 training job의 로그만 보는 것입니다.

In [ ]:
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

## 9. 학습 결과 확인

학습이 완료된 다음 S3에 저장된 산출물을 확인합니다.<br> model 결과물은 model.tar.gz에 저장되어 있고, 이외 학습 중 로그, 결과 산출물 등은 output.tar.gz에 저장할 수 있습니다.

In [ ]:
artifacts_dir = estimator.model_data.replace('model.tar.gz', '')
print(artifacts_dir)
!aws s3 ls --human-readable {artifacts_dir}

<br> S3에 저장된 학습 결과 산출물을 모두 노트북에 다운로드 받은 다음, 압축을 풉니다.

In [ ]:
model_dir = './autogluon_model'

!rm -rf $model_dir

import json , os

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

!aws s3 cp {artifacts_dir}model.tar.gz {model_dir}/model.tar.gz
!tar -xvzf {model_dir}/model.tar.gz -C {model_dir}

## 10. 학습 결과의 Visualization

In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
local_data_dir = './data/amzforecast'

In [ ]:
def prepare_dataset(data_dir):
    train_df = pd.read_csv(f'{data_dir}/target_train.csv')
    related_df = pd.read_csv(f'{data_dir}/related.csv')

    data = pd.merge(train_df, related_df, on=['timestamp', 'item_id'], how='left')
    data = data.sort_values(by=['item_id', 'timestamp']).reset_index(drop=True)
    
    train_data = TimeSeriesDataFrame.from_data_frame(
        data,
        id_column="item_id",
        timestamp_column="timestamp"
    )
    return train_data

In [ ]:
train_data = prepare_dataset(local_data_dir)

In [ ]:
predictor = TimeSeriesPredictor.load(path=model_dir)
predictor.leaderboard()

In [ ]:
train_data

In [ ]:
predictions = predictor.predict(train_data)
predictions

In [ ]:
def cal_metrics(pred, target):
    return {
        'MSE': ((pred - target) ** 2).mean(),
        'MAE': np.abs(pred - target).mean()
    }

In [ ]:
import matplotlib.pyplot as plt

# TimeSeriesDataFrame can also be loaded directly from a file
test_data = TimeSeriesDataFrame.from_path(f"{local_data_dir}/target_test.csv")

item_ids = ["casual", "registered"]
result = {}
for item_id in item_ids:
    plt.figure(figsize=(20, 3))
    
    y_past = train_data.loc[item_id]["demand"]
    y_pred = predictions.loc[item_id]
    y_test = test_data.loc[item_id]["demand"][:336]
    
    result[item_id] = cal_metrics(y_pred["mean"], y_test)

    plt.plot(y_past[-200:], label="Past time series values")
    plt.plot(y_pred["mean"], label="Mean forecast")
    plt.plot(y_test, label="Future time series values")

    plt.fill_between(
        y_pred.index, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval"
    )
    plt.legend()

In [ ]:
result